## Création dataframe 

In [1]:
import pandas as pd
from PIL import Image
import numpy as np
from scipy import ndimage

# Fonction pour compter les gros contours
def compter_gros_contours(image_path, seuil=130, proportion=0.5, aire_min=100):
    try:
        # Ouvrir l'image et convertir en niveaux de gris
        image = Image.open(image_path).convert('L')
        
        # Convertir l'image en tableau numpy
        image_np = np.array(image)
        
        # Découper la partie centrale de l'image
        h, w = image_np.shape
        centre_h = int(h * proportion)
        centre_w = int(w * proportion)
        debut_h = (h - centre_h) // 2
        debut_w = (w - centre_w) // 2
        image_centre = image_np[debut_h:debut_h + centre_h, debut_w:debut_w + centre_w]
        
        # Appliquer le seuillage
        image_seuillee = (image_centre > seuil).astype(np.uint8) * 255
        
        # Trouver les contours en utilisant les différences de gradients
        sx = ndimage.sobel(image_seuillee, axis=0, mode='constant')
        sy = ndimage.sobel(image_seuillee, axis=1, mode='constant')
        sobel = np.hypot(sx, sy)
        
        # Binariser le résultat du filtre Sobel
        contours = (sobel > 0).astype(np.uint8)
        
        # Étiqueter les objets (contours)
        labeled_array, num_features = ndimage.label(contours)
        
        # Filtrer les contours par leur aire
        areas = ndimage.sum(contours, labeled_array, range(1, num_features + 1))
        gros_contours = areas > aire_min
        nombre_gros_contours = np.sum(gros_contours)
        
        return nombre_gros_contours
    except Exception as e:
        print(f"Une erreur s'est produite pour l'image {image_path} : {e}")
        return None

# Créer une liste contenant vos DataFrames
liste_df = [df_ba, df_eos, df_ery, df_ig, df_lymph, df_mono, df_neutro, df_plat]

# Concaténer les DataFrames dans la liste
df = pd.concat(liste_df)
df = df.drop(columns=['Date création'])

# Ajouter une nouvelle colonne avec le nombre de gros contours
df['Gros contours'] = df['Nom fichier'].apply(lambda nom_fichier: compter_gros_contours(f"/path/to/your/images/directory/{nom_fichier}"))

# Afficher la DataFrame
print(df)

NameError: name 'df_ba' is not defined